# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Making-and-Viewing-data" data-toc-modified-id="Making-and-Viewing-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Making and Viewing data</a></div><div class="lev1 toc-item"><a href="#Defaults-kurfile-as-basis" data-toc-modified-id="Defaults-kurfile-as-basis-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Defaults kurfile as basis</a></div><div class="lev1 toc-item"><a href="#Use-50-chars-to-learn-1-char:-pytorch" data-toc-modified-id="Use-50-chars-to-learn-1-char:-pytorch-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Use 50 chars to learn 1 char: pytorch</a></div><div class="lev1 toc-item"><a href="#Use-50-chars-to-learn-1-char:-keras" data-toc-modified-id="Use-50-chars-to-learn-1-char:-keras-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Use 50 chars to learn 1 char: keras</a></div><div class="lev1 toc-item"><a href="#Model-use-gru-instead-of-lstm" data-toc-modified-id="Model-use-gru-instead-of-lstm-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Model use gru instead of lstm</a></div><div class="lev1 toc-item"><a href="#Use-50-chars-to-learn-another-50-chars" data-toc-modified-id="Use-50-chars-to-learn-another-50-chars-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Use 50 chars to learn another 50 chars</a></div>

## Making and Viewing data
- books/ for text files is at same level as this notebook
- download books/ with the following code

In [ ]:
!svn checkout https://github.com/deepgram/kur/trunk/examples/language-model/books

**Prepare Dataset**
- vocab = 30, meaning there are 30 unique chars, 26 letters, 4 symbols
- create cleaned.txt: remove all other symbols outside of 30 above
- create `data/` , inside we have `train.jsonl`, `validate.jsonl`, `evaluate.jsonl`, `test.jsonl` 
- seq_len = 50, meaning there are 50 timesteps as a sequence to predict next char or next 50 chars 
- `vocab` and `seq_len` are set inside `make_data.py` file

In [34]:
# create dataset, to change seq_len, go inside make_data.py to change directly
!python utils_func/make_data.py

# sample index is shown
# each dataset dimension is shown
# in each dataset section, first dim is in_seq, second dim is out_char

train :
sample Index: 0 13299
dims:
(13299, 50, 30)
(13299, 30)
validate :
sample Index: 13299 14130
dims:
(831, 50, 30)
(831, 30)
test :
sample Index: 14130 14961
dims:
(831, 50, 30)
(831, 30)
evaluate :
sample Index: 14961 15792
dims:
(831, 50, 30)
(831, 30)


**View Inputs and targets data**
- Let's see what the preprocessed data sample look like before entering model

In [35]:

!python utils_func/view_data.py



peek at train:
------
"the project gutenberg ebook of pride and prejudice" --> " "
{'in_seq': array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])}
in_seq --> out_char
{'out_char': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0])}
------
" particular resentment by his having slighted one " --> "o"
{'in_seq': array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])}
in_seq --> out_char
{'out_char': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0])}


peek at validate:
------
"particular resentment by his having slighted one o" --> "f"
{'in_seq': array([[0, 0, 0, ..., 0, 0, 0],
  

## Defaults kurfile as basis
- t1/defaults.yaml

In [68]:
%%writefile t1/defaults.yaml


# ANSWER: 
# solution1: error display when change from lstm to gru works 
# solution2: change backend to tensorflow backend, can help with the model loss is NaN issue too

---
                                                ##### setting section
settings:
    
  backend: pytorch    
    
    
    
  num_epochs: 1                    # leave it empty means inf number of epochs
                                 # so to use default value, just comment this line out
#   drop_neurons: 0.5
#   grad_clip: 5


         
                                                # 30 unique chars in the whole corpose
  vocab:                                         
    size: 30                          



                                                # 128 neurons for each rnn layer
                                                # 3 rnn layers in total
  rnn:
    size: 128                                    
    depth: 3                                     
        
        
        
  folder_name: t1  
    
  last_seq: no  

        
#####################################################################################


model:
  - input: in_seq


  - for:
      range: "{{ rnn.depth - 1 }}"
      iterate:
        - recurrent:
            size: "{{ rnn.size }}"
            type: "{{ rnn.type|default('lstm') }}"   #  lstm
                                                        # yes: return 30 characters on a sequence
            sequence: yes                             
            bidirectional: no
                                                        # -1: refer to last dimension
                                                        # however, no idea why last dim 
        - batch_normalization:
            axis: -1
#         - dropout: "{{drop_neurons}}"

  - recurrent:
      size: "{{ rnn.size }}"
      type: "{{ rnn.type|default('lstm') }}"   #  lstm
                                                        # no: just return the last character on a sequence
      sequence: "{{last_seq}}"                          
      bidirectional: no
  - batch_normalization
#   - dropout: "{{drop_neurons}}"
    
                                                        # now it is like 30 class-classification problem
                                                        # that's why we need 30 neurons here
  - dense: "{{vocab.size}}"                    
                                                 

  - activation: softmax

  - output: out_char                              
           
 ######################################################################################################       
        
                                                ##### loss section
            
            
                                                # use softmax and crossentropy to calc loss
                                                # target data is under variable `out_char`
loss:
  - target: out_char
    name: categorical_crossentropy

        
        
        
                                                ##### train section
        
train:
                                                
  data:
    - jsonl: data/train.jsonl
        
                                                
  epochs: "{{ num_epochs|default(5) }}"     
    
    
  weights:
    initial: "{{folder_name}}/best.w.kur"      
                                                # no need to save the best weights wrt training set
    last: "{{folder_name}}/last.w.kur"
        
        
                                                # produce and save info on loss for plotting loss vs batches...
  log: "{{folder_name}}/log"
    
    
  optimizer: 
    name: adam    
#     clip:                                              
#       norm: "{{grad_clip}}"   
        
                                                # save plot on loss vs batches on both training and validation set
  hooks:                                   
    - plot: "{{folder_name}}/loss.png"
        
                               
                                                ##### validate section

validate:
    
  data:
    - jsonl: data/validate.jsonl
        
  weights: "{{folder_name}}/best.w.kur"

                                                ##### test section
test:
    
  data:
    - jsonl: data/test.jsonl
        
  weights: 
    initial: "{{folder_name}}/best.w.kur"

                                                ##### evaluate section
evaluate:
    
  data:
    - jsonl: data/evaluate.jsonl
        
  weights: 
    initial: "{{folder_name}}/best.w.kur"
                                                # use output_hook to produce predictions along with true labels or targets 
  destination: "{{folder_name}}/output.pkl"

Overwriting t1/defaults.yaml


## Use 50 chars to learn 1 char: pytorch
**Attention**
- use pytoch
- train 2 epochs

**Question!**
- input sample dim is correct (50, 30), but
- The model layer dimension look strange, see `kur -vv build ` output below, especially the number `512`

In [47]:
%%writefile t1/pytorch_50_1.yaml

---
settings: 
  num_epochs: 2
    


        

include: defaults.yaml
...

Writing t1/pytorch_50_1.yaml


In [48]:
!kur -vv build t1/pytorch_50_1.yaml

[INFO 2017-03-09 19:38:43,087 kur.kurfile:699] Parsing source: t1/pytorch_50_1.yaml, included by top-level.
[INFO 2017-03-09 19:38:43,090 kur.kurfile:699] Parsing source: defaults.yaml, included by t1/pytorch_50_1.yaml.
[INFO 2017-03-09 19:38:43,108 kur.kurfile:82] Parsing Kurfile...
[DEBUG 2017-03-09 19:38:43,108 kur.kurfile:784] Parsing Kurfile section: settings
[DEBUG 2017-03-09 19:38:43,109 kur.kurfile:784] Parsing Kurfile section: train
[DEBUG 2017-03-09 19:38:43,115 kur.kurfile:784] Parsing Kurfile section: validate
[DEBUG 2017-03-09 19:38:43,116 kur.kurfile:784] Parsing Kurfile section: test
[DEBUG 2017-03-09 19:38:43,118 kur.kurfile:784] Parsing Kurfile section: evaluate
[DEBUG 2017-03-09 19:38:43,122 kur.containers.layers.placeholder:63] Using short-hand name for placeholder: in_seq
[DEBUG 2017-03-09 19:38:43,123 kur.containers.layers.placeholder:97] Placeholder "in_seq" has a deferred shape.
[DEBUG 2017-03-09 19:38:43,137 kur.containers.layers.output:50] Using short-hand name

## Use 50 chars to learn 1 char: keras
**Attention**
- use keras
- train 2 epochs
- `kur -vv build ` output has no number `512`, and looks more intuitive


In [51]:
%%writefile t2/keras_50_1.yaml

---
settings: 
  backend: keras

  folder_name: t2
include: ../t1/defaults.yaml
...    

Writing t2/keras_50_1.yaml


In [52]:
!kur -vv build t2/keras_50_1.yaml

[INFO 2017-03-09 19:44:57,691 kur.kurfile:699] Parsing source: t2/keras_50_1.yaml, included by top-level.
[INFO 2017-03-09 19:44:57,694 kur.kurfile:699] Parsing source: ../t1/defaults.yaml, included by t2/keras_50_1.yaml.
[INFO 2017-03-09 19:44:57,710 kur.kurfile:82] Parsing Kurfile...
[DEBUG 2017-03-09 19:44:57,710 kur.kurfile:784] Parsing Kurfile section: settings
[DEBUG 2017-03-09 19:44:57,711 kur.kurfile:784] Parsing Kurfile section: train
[DEBUG 2017-03-09 19:44:57,715 kur.kurfile:784] Parsing Kurfile section: validate
[DEBUG 2017-03-09 19:44:57,716 kur.kurfile:784] Parsing Kurfile section: test
[DEBUG 2017-03-09 19:44:57,717 kur.kurfile:784] Parsing Kurfile section: evaluate
[DEBUG 2017-03-09 19:44:57,721 kur.containers.layers.placeholder:63] Using short-hand name for placeholder: in_seq
[DEBUG 2017-03-09 19:44:57,722 kur.containers.layers.placeholder:97] Placeholder "in_seq" has a deferred shape.
[DEBUG 2017-03-09 19:44:57,730 kur.containers.layers.output:50] Using short-hand na

**Train with 1 epochs**
**Attention**
- to train lstm layers
- pytorch is fast
- if use keras, must use tensorflow backend
- but very slow compared to pytorch

In [59]:
%%writefile t2/keras_50_1.yaml

---
settings: 
  backend: 
    name: keras
    backend: tensorflow

  num_epochs: 1  
    
  folder_name: t2
include: ../t1/defaults.yaml
...    

Overwriting t2/keras_50_1.yaml


In [60]:
!kur -v train t2/keras_50_1.yaml

[INFO 2017-03-09 19:50:28,907 kur.kurfile:699] Parsing source: t2/keras_50_1.yaml, included by top-level.
[INFO 2017-03-09 19:50:28,911 kur.kurfile:699] Parsing source: ../t1/defaults.yaml, included by t2/keras_50_1.yaml.
[INFO 2017-03-09 19:50:28,927 kur.kurfile:82] Parsing Kurfile...
[INFO 2017-03-09 19:50:28,964 kur.loggers.binary_logger:71] Loading log data: t2/log
[INFO 2017-03-09 19:50:34,751 kur.backend.backend:80] Creating backend: keras
[INFO 2017-03-09 19:50:34,751 kur.backend.backend:83] Backend variants: none
[INFO 2017-03-09 19:50:34,751 kur.backend.keras_backend:81] The tensorflow backend for Keras has been requested.
[INFO 2017-03-09 19:50:36,030 kur.backend.keras_backend:195] Keras is loaded. The backend is: tensorflow
[INFO 2017-03-09 19:50:36,030 kur.model.model:261] Enumerating the model containers.
[INFO 2017-03-09 19:50:36,030 kur.model.model:266] Assembling the model dependency graph.
[INFO 2017-03-09 19:50:36,031 kur.model.model:281] Connecting the model graph.
[

**Evaluating and Viewing result**
- evaluate once and output.pkl is produced
- use view_output function to see how well trained model predict next char
    - view_output will compare prediction with targets
    - print them in comparison nicely

In [62]:
!kur -v evaluate t2/keras_50_1.yaml

[INFO 2017-03-09 19:56:32,202 kur.kurfile:699] Parsing source: t2/keras_50_1.yaml, included by top-level.
[INFO 2017-03-09 19:56:32,205 kur.kurfile:699] Parsing source: ../t1/defaults.yaml, included by t2/keras_50_1.yaml.
[INFO 2017-03-09 19:56:32,221 kur.kurfile:82] Parsing Kurfile...
[INFO 2017-03-09 19:56:32,570 kur.backend.backend:80] Creating backend: keras
[INFO 2017-03-09 19:56:32,570 kur.backend.backend:83] Backend variants: none
[INFO 2017-03-09 19:56:32,570 kur.backend.keras_backend:81] The tensorflow backend for Keras has been requested.
[INFO 2017-03-09 19:56:33,648 kur.backend.keras_backend:195] Keras is loaded. The backend is: tensorflow
[INFO 2017-03-09 19:56:33,648 kur.model.model:261] Enumerating the model containers.
[INFO 2017-03-09 19:56:33,648 kur.model.model:266] Assembling the model dependency graph.
[INFO 2017-03-09 19:56:33,648 kur.model.model:281] Connecting the model graph.
[INFO 2017-03-09 19:56:34,751 kur.model.model:285] Model inputs:  in_seq
[INFO 2017-03

In [63]:
# viewing result from evaluation: 
!python utils_func/view_outputs.py 't2/output.pkl'

"ing your time with me. mr. bingley followed his ad" --> " "
                                                       INCORRECT (v)
"ng your time with me. mr. bingley followed his adv" --> " "
                                                       INCORRECT (i)
"g your time with me. mr. bingley followed his advi" --> " "
                                                       INCORRECT (c)
" your time with me. mr. bingley followed his advic" --> " "
                                                       INCORRECT (e)
"your time with me. mr. bingley followed his advice" --> " "
                                                       INCORRECT (.)
"our time with me. mr. bingley followed his advice." --> " "
                                                       CORRECT
"ur time with me. mr. bingley followed his advice. " --> " "
                                                       INCORRECT (m)
"r time with me. mr. bingley followed his advice. m" --> " "
                                   

**Testing**

In [65]:
!kur -v test t2/keras_50_1.yaml

[INFO 2017-03-09 19:59:48,605 kur.kurfile:699] Parsing source: t2/keras_50_1.yaml, included by top-level.
[INFO 2017-03-09 19:59:48,609 kur.kurfile:699] Parsing source: ../t1/defaults.yaml, included by t2/keras_50_1.yaml.
[INFO 2017-03-09 19:59:48,625 kur.kurfile:82] Parsing Kurfile...
[INFO 2017-03-09 19:59:48,975 kur.backend.backend:80] Creating backend: keras
[INFO 2017-03-09 19:59:48,975 kur.backend.backend:83] Backend variants: none
[INFO 2017-03-09 19:59:48,975 kur.backend.keras_backend:81] The tensorflow backend for Keras has been requested.
[INFO 2017-03-09 19:59:50,200 kur.backend.keras_backend:195] Keras is loaded. The backend is: tensorflow
[INFO 2017-03-09 19:59:50,200 kur.model.model:261] Enumerating the model containers.
[INFO 2017-03-09 19:59:50,200 kur.model.model:266] Assembling the model dependency graph.
[INFO 2017-03-09 19:59:50,201 kur.model.model:281] Connecting the model graph.
[INFO 2017-03-09 19:59:51,350 kur.model.model:285] Model inputs:  in_seq
[INFO 2017-03

## Model use gru instead of lstm

In [79]:
%%writefile t3/gru_theano.yaml

---
settings:
  backend: keras
    
  folder_name: t3
  rnn:
    type: gru
        
  num_epochs: 2

include: ../t1/defaults.yaml     
...

Overwriting t3/gru_theano.yaml


In [76]:
!kur -vv build t3/gru_theano.yaml

[INFO 2017-03-09 20:57:13,677 kur.kurfile:699] Parsing source: t3/gru_theano.yaml, included by top-level.
[INFO 2017-03-09 20:57:13,681 kur.kurfile:699] Parsing source: ../t1/defaults.yaml, included by t3/gru_theano.yaml.
[INFO 2017-03-09 20:57:13,699 kur.kurfile:82] Parsing Kurfile...
[DEBUG 2017-03-09 20:57:13,700 kur.kurfile:784] Parsing Kurfile section: settings
[DEBUG 2017-03-09 20:57:13,702 kur.kurfile:784] Parsing Kurfile section: train
[DEBUG 2017-03-09 20:57:13,708 kur.kurfile:784] Parsing Kurfile section: validate
[DEBUG 2017-03-09 20:57:13,710 kur.kurfile:784] Parsing Kurfile section: test
[DEBUG 2017-03-09 20:57:13,712 kur.kurfile:784] Parsing Kurfile section: evaluate
[DEBUG 2017-03-09 20:57:13,716 kur.containers.layers.placeholder:63] Using short-hand name for placeholder: in_seq
[DEBUG 2017-03-09 20:57:13,717 kur.containers.layers.placeholder:97] Placeholder "in_seq" has a deferred shape.
[DEBUG 2017-03-09 20:57:13,738 kur.containers.layers.output:50] Using short-hand na

**How much faster with gru**
- 3 times faster

In [80]:
!kur -v train t3/gru_theano.yaml

[INFO 2017-03-09 21:05:25,595 kur.kurfile:699] Parsing source: t3/gru_theano.yaml, included by top-level.
[INFO 2017-03-09 21:05:25,599 kur.kurfile:699] Parsing source: ../t1/defaults.yaml, included by t3/gru_theano.yaml.
[INFO 2017-03-09 21:05:25,617 kur.kurfile:82] Parsing Kurfile...
[INFO 2017-03-09 21:05:25,643 kur.loggers.binary_logger:107] Log does not exist. Creating path: t3/log
[INFO 2017-03-09 21:05:32,554 kur.backend.backend:80] Creating backend: keras
[INFO 2017-03-09 21:05:32,555 kur.backend.backend:83] Backend variants: none
[INFO 2017-03-09 21:05:32,555 kur.backend.keras_backend:122] No particular backend for Keras has been requested.
[INFO 2017-03-09 21:05:33,464 kur.backend.keras_backend:195] Keras is loaded. The backend is: theano
[INFO 2017-03-09 21:05:33,464 kur.model.model:261] Enumerating the model containers.
[INFO 2017-03-09 21:05:33,465 kur.model.model:266] Assembling the model dependency graph.
[INFO 2017-03-09 21:05:33,465 kur.model.model:281] Connecting the 

**QUESTION: How to generate long text** like karpathy did in [his post](https://hyp.is/KYLatAPxEee89XOhgBMapQ/karpathy.github.io/2015/05/21/rnn-effectiveness/)?
- What I did above is similar to graph 1 on the left (SEE figure below), I use a model use 30 characters (index 1 to index 30) to predict the next (31th) character (in the graph I used 50, just so you know)
- What I want to achieve in the next kurfile below is graph 2 in the middle, which is a model to use 30 characters (say index 1 to 30th) to predict 30 characters with index (2 to 31th)
- to generate a long text like what karpathy did in the post link above, should I train my model like graph2 with much longer seq_len, say 200? Is it how karpathy got the long text??



In [1]:
from IPython.display import Image
Image(url='https://lh3.googleusercontent.com/Ybix_t5PBJaST4icd_BY8WvvFRfieba58Wkup2S1gpRf3mFdxp00iOtpWKdh6ptM9uexoTORCr8mfQL4Qy4oeiIzIZxtGFXgZsSA34g3teyBhJ_fxqQphwxpPyCsa7q4t530EE4BT8XiceQwoAuCl1PhBFR-2UA4NfZZiL5hyrVeXOdoRSzZ1jl45MekOaHad-9j1A5_PNMeOZz8BmLx3Uc5xFWNrdlZ0rSql385Sum81rdntmMUWY8MnN7LYzr96IlgN3Hxd0uQZkbc5pn6YD8LTew_7xlSGBiYxqIoc2qfjmFd_mWr-RP445kq_firczCmjHows4aOsKw0yK-DaOBCGc-ovUv9c_1OFqYQYKGHk6VTMf3TF3gP4X2g0tOk4K6zHP3MSTZGpvEj1-HGCq8kVPwBXVnj1jI-nKnLwIiVePlbYt8qAls4uhoOXGJb5Nm4gotIoOTsBdR1NRYiULuUghcIjbWy22uCA-2CbO59oOLf-dXXHO9brhOutFUP5Mazt3BQk6SOUQbf7tqbqsRWlp7iMXdC0kwXEeUiNGSw0caNKOPjhg2z-GBAqDoCrmuc9YZQ1UyI-UZr-ppyNLVXaecRfjePylfNyp1eFjx8p2ccJ5xo=w1912-h1352-no')

## Use 50 chars to learn another 50 chars

**Error**
- see below

**Possible solutions**
- [one](https://github.com/deepgram/kur/issues/13)
- [another](https://github.com/deepgram/kur/issues/23)

**keras-theano**

In [87]:
%%writefile t4/theano_50_50.yaml

---

settings: 
  backend: keras
    
  folder_name: t4  
    
  last_seq: yes  
    
  rnn:
    type: gru
    
  num_epochs: 1                    # leave it empty means inf number of epochs
                                 # so to use default value, just comment this line out
#   drop_neurons: 0.5
#   grad_clip: 5


model:
  - input: in_seq


  - for:
      range: "{{ rnn.depth - 1 }}"
      iterate:
        - recurrent:
            size: "{{ rnn.size }}"
            type: "{{ rnn.type|default('lstm') }}"   #  lstm
                                                        # yes: return 30 characters on a sequence
            sequence: yes                             
            bidirectional: no
                                                        # -1: refer to last dimension
                                                        # however, no idea why last dim 
        - batch_normalization:
            axis: -1
#         - dropout: "{{drop_neurons}}"

  - recurrent:
      size: "{{ rnn.size }}"
      type: "{{ rnn.type|default('lstm') }}"   #  lstm
                                                        # no: just return the last character on a sequence
      sequence: "{{last_seq}}"                          
      bidirectional: no
  - batch_normalization
#   - dropout: "{{drop_neurons}}"

  
                                                   # http://kur.deepgram.com/containers.html?highlight=parallel#parallel
                                                   # Question!: how to understand the following code? 
                                                   # - apply a dense layer of 30 neurons to each char of the sequence (seq_len = 30)
                                                   # seq_len is 30 defined when making data
  - parallel:
      apply:
        - dense: "{{vocab.size}}"


  
  - activation: softmax

  - output: out_char                              # make a name of output layer
           

include: ../t1/defaults.yaml
...

Overwriting t4/theano_50_50.yaml


**Build first**: Questions!
- fails
- keras provide sensible error message: `ValueError: Input dimension mis-match. (input[0].shape[1] = 30, input[1].shape[1] = 50)`
- pytorch does not below

In [88]:
!kur -vv build t4/theano_50_50.yaml

[INFO 2017-03-09 21:12:25,093 kur.kurfile:699] Parsing source: t4/theano_50_50.yaml, included by top-level.
[INFO 2017-03-09 21:12:25,105 kur.kurfile:699] Parsing source: ../t1/defaults.yaml, included by t4/theano_50_50.yaml.
[INFO 2017-03-09 21:12:25,124 kur.kurfile:82] Parsing Kurfile...
[DEBUG 2017-03-09 21:12:25,124 kur.kurfile:784] Parsing Kurfile section: settings
[DEBUG 2017-03-09 21:12:25,126 kur.kurfile:784] Parsing Kurfile section: train
[DEBUG 2017-03-09 21:12:25,132 kur.kurfile:784] Parsing Kurfile section: validate
[DEBUG 2017-03-09 21:12:25,134 kur.kurfile:784] Parsing Kurfile section: test
[DEBUG 2017-03-09 21:12:25,137 kur.kurfile:784] Parsing Kurfile section: evaluate
[DEBUG 2017-03-09 21:12:25,142 kur.containers.layers.placeholder:63] Using short-hand name for placeholder: in_seq
[DEBUG 2017-03-09 21:12:25,143 kur.containers.layers.placeholder:97] Placeholder "in_seq" has a deferred shape.
[DEBUG 2017-03-09 21:12:25,173 kur.containers.layers.output:50] Using short-han

**Using pytorch**     
Questions!:     
- no error reported in building models, but has strange dim number 384 in debug information
- training fails, but error message is not helpful below

In [90]:
%%writefile t5/pytorch_50_50.yaml

---
settings: 
  folder_name: t5

  backend: pytorch
    
include: ../t4/theano_50_50.yaml

Writing t5/pytorch_50_50.yaml


In [91]:
!kur -vv build t5/pytorch_50_50.yaml

[INFO 2017-03-09 21:16:04,847 kur.kurfile:699] Parsing source: t5/pytorch_50_50.yaml, included by top-level.
[INFO 2017-03-09 21:16:04,850 kur.kurfile:699] Parsing source: ../t4/theano_50_50.yaml, included by t5/pytorch_50_50.yaml.
[INFO 2017-03-09 21:16:04,858 kur.kurfile:699] Parsing source: ../t1/defaults.yaml, included by t5/../t4/theano_50_50.yaml.
[INFO 2017-03-09 21:16:04,875 kur.kurfile:82] Parsing Kurfile...
[DEBUG 2017-03-09 21:16:04,875 kur.kurfile:784] Parsing Kurfile section: settings
[DEBUG 2017-03-09 21:16:04,877 kur.kurfile:784] Parsing Kurfile section: train
[DEBUG 2017-03-09 21:16:04,882 kur.kurfile:784] Parsing Kurfile section: validate
[DEBUG 2017-03-09 21:16:04,883 kur.kurfile:784] Parsing Kurfile section: test
[DEBUG 2017-03-09 21:16:04,884 kur.kurfile:784] Parsing Kurfile section: evaluate
[DEBUG 2017-03-09 21:16:04,887 kur.containers.layers.placeholder:63] Using short-hand name for placeholder: in_seq
[DEBUG 2017-03-09 21:16:04,888 kur.containers.layers.placehol

In [92]:
!kur -v train t5/pytorch_50_50.yaml

[INFO 2017-03-09 21:18:16,974 kur.kurfile:699] Parsing source: t5/pytorch_50_50.yaml, included by top-level.
[INFO 2017-03-09 21:18:16,978 kur.kurfile:699] Parsing source: ../t4/theano_50_50.yaml, included by t5/pytorch_50_50.yaml.
[INFO 2017-03-09 21:18:16,990 kur.kurfile:699] Parsing source: ../t1/defaults.yaml, included by t5/../t4/theano_50_50.yaml.
[INFO 2017-03-09 21:18:17,008 kur.kurfile:82] Parsing Kurfile...
[INFO 2017-03-09 21:18:17,032 kur.loggers.binary_logger:107] Log does not exist. Creating path: t5/log
[INFO 2017-03-09 21:18:22,713 kur.backend.backend:80] Creating backend: pytorch
[INFO 2017-03-09 21:18:22,713 kur.backend.backend:83] Backend variants: none
[INFO 2017-03-09 21:18:22,860 kur.model.model:261] Enumerating the model containers.
[INFO 2017-03-09 21:18:22,860 kur.model.model:266] Assembling the model dependency graph.
[INFO 2017-03-09 21:18:22,860 kur.model.model:281] Connecting the model graph.
[{'shape': (50, 128), 'layer': <function TorchModel.add_operation